In [12]:
YOUR_NAME = "Aleks" 

In [1]:
# https://keras.io/examples/vision/mnist_convnet/
# https://www.kaggle.com/code/shivanisheth2911/lstm-keras-on-mnist

In [2]:
import keras
from keras import layers
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

## Dataset

In [3]:
input_shape = (28, 28, 1)
num_classes = 10

In [4]:
# Dataset Size
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Make Models

In [5]:
def make_cnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [6]:
def make_fnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Dense(num_classes, activation="relu"),
            layers.Dense(num_classes, activation="relu"),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [7]:
def make_rnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            SimpleRNN(128, return_sequences=False,
                    input_shape=(input_shape[0], input_shape[1]),
                    unroll=True),
            layers.Dense(num_classes, activation="relu"),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

## Experiment

In [8]:
def fit_and_validate(model_function, model_name, epochs, x_train, y_train, x_test, y_test, df, input_shape, num_classes):
    model = model_function(input_shape, num_classes)
    batch_size = 128

    start_time = time.time()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)
    time_to_train = time.time() - start_time

    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    start_time = time.time()
    model.predict(x_test, verbose=0)
    time_to_predict = time.time() - start_time
    
    start_time = time.time()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)
    time_to_retrain = time.time() - start_time

    df.loc[len(df)] = [model_name, batch_size, epochs, len(x_train), time_to_train, time_to_retrain, time_to_predict, score[1], YOUR_NAME]
    return df

In [9]:
# Action Space
all_epochs = [1]
all_models = {
    "RNN": make_rnn,
    "FNN": make_fnn,
    "CNN": make_cnn
}
all_dataset_sizes = [i/100 for i in range(25, 105, 1)]

In [10]:
# Data Collection
df = pd.DataFrame(columns=['model', 'batch_size', 'epochs', 'dataset_size', 'time_to_train', 'time_to_retrain', 'time_to_predict', 'accuracy', 'name'])

In [11]:
for i in range(10):
    for epochs in all_epochs:
        for dataset_size in all_dataset_sizes:
            for model in all_models:
                print(f"{i}/10  {epochs}  {model}  {dataset_size}")
                size = int(len(x_train) * dataset_size)
    
                subset_x_train = x_train[:size]
                subset_y_train = y_train[:size]
                subset_x_test = x_test[:size]
                subset_y_test = y_test[:size]
                
                df = fit_and_validate(all_models[model], model, epochs, subset_x_train, subset_y_train, subset_x_test, subset_y_test, df, input_shape, num_classes)
                df.to_csv(f"{YOUR_NAME}_results.csv")

0/10  1  RNN  0.25


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Test loss: 1.1520148515701294
Test accuracy: 0.7055000066757202
0/10  1  FNN  0.25
Test loss: 0.40625229477882385
Test accuracy: 0.8894000053405762
0/10  1  CNN  0.25
Test loss: 0.22330652177333832
Test accuracy: 0.9375
0/10  1  RNN  0.26


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Test loss: 1.168779969215393
Test accuracy: 0.761900007724762
0/10  1  FNN  0.26
Test loss: 0.45029139518737793
Test accuracy: 0.8795999884605408
0/10  1  CNN  0.26
Test loss: 0.24850089848041534
Test accuracy: 0.9289000034332275
0/10  1  RNN  0.27


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Test loss: 1.060890793800354
Test accuracy: 0.7738999724388123
0/10  1  FNN  0.27
Test loss: 0.3975954055786133
Test accuracy: 0.8878999948501587
0/10  1  CNN  0.27
Test loss: 0.22877682745456696
Test accuracy: 0.9358999729156494



KeyboardInterrupt



In [ ]:
df